# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2

### opencc是繁簡轉換工具

In [ ]:
!pip install opencc-python-reimplemented

### gensim是訓練word2vec的函式庫

In [ ]:
# 从 gensim 库中导入 WikiCorpus 类
from gensim.corpora import WikiCorpus

# 使用 WikiCorpus 类加载维基百科的语料库文件，并创建一个 WikiCorpus 对象
wiki_corpus = WikiCorpus('zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})


In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
# 打开一个文本文件用于写入，编码方式为 UTF-8
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    # 遍历维基百科语料库中的每篇文章
    for text in wiki_corpus.get_texts():
        # 将文章中的每个词用空格连接起来，然后写入文件，并在每篇文章末尾添加换行符
        f.write(' '.join(text)+'\n')

        # 文章数量加一
        text_num += 1

        # 每处理完 10000 篇文章就打印一次处理进度
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    # 打印最终的处理进度，即文章总数
    print('{} articles processed.'.format(text_num))


In [ ]:
# 导入所需库
import jieba
from opencc import OpenCC

# 初始化 OpenCC，将繁体中文转换为简体中文
cc = OpenCC('s2t')

# 读取文件中的训练数据（繁体中文）
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()

# 将繁体中文转换为简体中文
train_data = cc.convert(train_data)

# 使用 jieba 进行分词
train_data = jieba.lcut(train_data)

# 去除空格等无效词语
train_data = [word for word in train_data if word != '']

# 将分词结果转换为字符串形式
train_data = ' '.join(train_data)

# 将处理后的文本写入到新文件中
open('seg.txt', 'w', encoding='utf-8').write(train_data)


In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [ ]:
# 导入 Word2Vec 相关模块
from gensim.models import word2vec

# 设置 Word2Vec 训练参数
# Settings
seed = 666               # 随机种子
sg = 0                   # 选择 CBOW 模型（sg=0），或者 Skip-gram 模型（sg=1）
window_size = 10         # 上下文窗口大小
#vector_size = 100
min_count = 1            # 忽略词频低于 min_count 的词语
workers = 8              # 使用多少个 CPU 核心进行训练
#epochs = 5
batch_words = 10000      # 每个批次包含的词语数量

# 从分词后的文本文件中读取训练数据
train_data = word2vec.LineSentence('seg.txt')

# 使用 Word2Vec 模型进行训练
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

# 保存训练好的 Word2Vec 模型
model.save('word2vec.model')


In [ ]:
# 导入 Word2Vec 模块
from gensim.models import word2vec

# 要查询相似词语的词语
string = '電腦'

# 加载之前训练好的 Word2Vec 模型
model = word2vec.Word2Vec.load('word2vec.model')

# 打印要查询相似词语的原词
print(string)

# 遍历模型中与原词最相似的词语，并打印出来
for item in model.wv.most_similar(string):
    print(item)
